# Pre-processing YOLO labels
<sub>Author: [faridjn</sub>](https://github.com/faridjn)

## Libraries

In [1]:
#!pip install shapely

### Import necessary libraries

In [2]:
#fundamental
import pandas as pd
import json

#OS and sys
import os

#labels geometry
from shapely.geometry import box
from PIL import Image
from shapely.geometry import Polygon

## Utility Functions

In [3]:
def get_files_with_extension(folder, extension):    
    '''a function that takes a folder path and a file extension as inputs, 
    and returns a list of all files in the folder with the specified extension,
    case-insensitive. It can be used for quickly obtaining a list of files
    with a specific file type from a directory'''
    # specify the desired file extension
    extension = extension.lower()
    # get a list of all files in the folder
    files = os.listdir(folder)
    # filter out only the files with the desired extension
    files_with_extention = [f for f in files if f.lower().endswith(extension)]
    # print the list of desired files
    return files_with_extention

def get_image_read_path_and_size(image_file):
    imgpil = Image.open(image_file)
    wd, ht = imgpil.size
    return wd, ht


def resize_image(image_file, OUT_IMGSZ):
    imgpil = Image.open(image_file)
    wd, ht = imgpil.size
    scale_factor = OUT_IMGSZ / max(wd, ht)
    img_resized = imgpil.resize((int(wd * scale_factor), int(ht * scale_factor)), Image.LANCZOS)
    return img_resized, scale_factor


def save_resized_image(img_resized, image_write_path, img):
    img_resized.save(image_write_path + '\\' + img)


def get_image_object(annotations, img):
    filt = annotations.index.str.lower() == img.lower()
    if filt.sum() == 1:
        return annotations.loc[filt]
    return None


def process_label_data(img_obj):
    lbl = img_obj['label'].iloc[0]
    lbl = lbl.replace("\'", "\"")
    return json.loads(lbl)


def create_label_file(label_write_path, label_file_name):
    return open(label_write_path + label_file_name, 'w')


# polygon = Polygon([(0, 0), (1, 1), (1, 0)])
def polygon_to_bbox(polygon_cords):
    '''Args: List of Polygon_cords '''
    polygon = Polygon(polygon_cords)
    return polygon.bounds


def process_objects(lbl, wd, ht, f1, scale_factor):
    for obj in lbl['objects']:
        polygon_coords_list = []

        polygon_coords = obj.get('polygon')
        if polygon_coords is not None:
            for pnt in polygon_coords:
                polygon_coords_list.append(tuple(pnt.values()))

            polygon_coords_list = [(int(x * scale_factor), int(y * scale_factor)) for x, y in polygon_coords_list]

            bbox = polygon_to_bbox(polygon_coords_list)
            minx, miny, maxx, maxy = bbox

            bbox_s = box(minx, miny, maxx, maxy)

            object_annos_line = str(CLASS_DICT[obj['value']]) + ' ' \
                                + str(bbox_s.centroid.x / (wd * scale_factor)) + ' ' \
                                + str(bbox_s.centroid.y / (ht * scale_factor)) + ' ' \
                                + str((maxx - minx) / (wd * scale_factor)) + ' ' \
                                + str((maxy - miny) / (ht * scale_factor)) + '\n'

            f1.write(object_annos_line)


def process_images(all_images, annotations, image_read_path, label_write_path, image_write_path, OUT_IMGSZ):

    for img in all_images:

        print(img)
        image_file = image_read_path + '\\' + img
        wd, ht = get_image_read_path_and_size(image_file)

        img_resized, scale_factor = resize_image(image_file, OUT_IMGSZ)
        save_resized_image(img_resized, image_write_path, img)

        img_obj = get_image_object(annotations, img)
        if img_obj is not None:
            lbl = process_label_data(img_obj)

            label_file_name = img.split('.')[0] + '.txt'
            with create_label_file(label_write_path, label_file_name) as f1:
                process_objects(lbl, wd, ht, f1, scale_factor)

## Settings

In [4]:
# Label dictionary
CLASS_DICT = {'crossarm':0,
              'cutouts':1,
              'insulator':2,
              'pole':3,
              'transformers':4,
              'background_structure':5}

In [5]:
#image size
OUT_IMGSZ = 1024

In [6]:
# Define root directory
ROOT_PATH = os.path.normpath(os.getcwd() + os.sep + os.pardir)

## Read Annotation

In [7]:
# Annotation file
annotation_csv = ROOT_PATH + '\\data\\raw\\annotation\\' + 'Overhead-Distribution-Labels.csv'

In [8]:
#Read annotations
annotations = pd.read_csv(annotation_csv)

In [9]:
#prepare annotation dataframe
annotations.columns = ['label', 'image']
annotations['image'] = annotations['image'].str.lower()
annotations.set_index(['image'], inplace=True)
annotations.head()

,label
image,
1 (1).jpg,"{'objects': [{'value': 'insulator', 'polygon':..."
1 (10).jpg,"{'objects': [{'value': 'other_wire', 'line': [..."
1 (100).jpg,"{'objects': [{'value': 'insulator', 'polygon':..."
1 (1000).jpg,"{'objects': [{'value': 'conductor', 'line': [{..."
1 (1001).jpg,"{'objects': [{'value': 'insulator', 'polygon':..."


## Process labels

In [10]:
# Choose dataset "test|train|val"
DATASETS = ['test', 'train', 'val']

print('Processing labels...')

for DATASET in DATASETS:
    
    print('###################### ' + DATASET + ' #############################')
        
    ############## READ IMAGES #############
    # Define image and label read folders
    image_read_path= ROOT_PATH + '\\data\\intermdiate\\' + DATASET + '\\images\\'
    # Display reading images
    all_images = get_files_with_extension(image_read_path, 'jpg')
    
    ############## WRITE FODLER ##############
    # Define image and label write folders
    image_write_path = ROOT_PATH + '\\data\\processed\\' + DATASET + '\\images\\'
    label_write_path = ROOT_PATH + '\\data\\processed\\' + DATASET + '\\labels\\'
    
    
    ############## CREATE FODLERS ##############
    #Create the folder if it doesn't exist
    if not os.path.exists(image_write_path):
        os.makedirs(image_write_path, exist_ok=True)        
    if not os.path.exists(label_write_path):
        os.makedirs(label_write_path, exist_ok=True)

    ############## PROCESS LABELS ##############
    process_images(all_images, annotations, image_read_path, label_write_path, image_write_path, OUT_IMGSZ)

Processing labels...
###################### test #############################
1 (113).JPG
1 (1184).JPG
1 (1299).JPG
1 (1385).JPG
1 (1490).JPG
1 (1653).JPG
1 (1671).JPG
1 (1681).JPG
1 (1834).JPG
1 (451).JPG
1 (475).JPG
1 (515).JPG
1 (54).JPG
1 (574).JPG
1 (818).JPG
###################### train #############################
1 (101).JPG
1 (1044).JPG
1 (1047).JPG
1 (1053).JPG
1 (1057).JPG
1 (1078).JPG
1 (1143).JPG
1 (1159).JPG
1 (1169).JPG
1 (1170).JPG
1 (1177).JPG
1 (1187).JPG
1 (1203).JPG
1 (1228).JPG
1 (1255).JPG
1 (1285).JPG
1 (1292).JPG
1 (1365).JPG
1 (1395).JPG
1 (1400).JPG
1 (1404).JPG
1 (141).JPG
1 (1427).JPG
1 (145).JPG
1 (1486).JPG
1 (1505).JPG
1 (1510).JPG
1 (1539).JPG
1 (1619).JPG
1 (1625).JPG
1 (1632).JPG
1 (1660).JPG
1 (1697).JPG
1 (17).JPG
1 (1772).JPG
1 (1777).JPG
1 (1778).JPG
1 (1826).JPG
1 (1845).JPG
1 (210).JPG
1 (269).JPG
1 (285).JPG
1 (298).JPG
1 (314).JPG
1 (366).JPG
1 (369).JPG
1 (39).JPG
1 (393).JPG
1 (42).JPG
1 (439).JPG
1 (459).JPG
1 (478).JPG
1 (527).JPG
1 (567)